#### Import

In [1]:
import os
import glob
import sys

import shutil
import json

import numpy as np
import random
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import nibabel as nib
from scipy.ndimage import morphology
import SimpleITK
import pandas as pd

### Interactive scribble and graphcut Plan
#### Choose (a) Patient name, srcDir, expDir, patConfig, graphCut config (b) Create random scribbles; (c) Load in slicer : Volumes (CT, PET, softmax), Segmentations (ground Truth:  gt, prediction from NN: predNN, foreground scribble: fgScribble, background-scribble: bgScribble, initial graph cut results using CT, PET and Softmax) (d) User modifies fgScribble and bgSScribble using slicer (e) Use Jupyter NB to save updated fgScribble and bgScribble (f) Repeat graphcut using jupyter - graph cut results using CT, PET and Softmax; (f) View in slice -> go back to step(d)

### Helper functions 
#### Most of these has already been tested in other notebook(s); 

In [2]:
sys.path.append('/home/user/DMML/CodeAndRepositories/MMGTVSeg')
import src
from src import scribbleHelper, gcHelper, graphCutClient, graphCutServer, libclient, libserver
from src.scribbleHelper import checkFolderExistenceAndCreate, readAndScaleImageData,\
    dice_coef_func, dice_multi_label, disk, ball, bbox2_3D, getUnionBoundingBoxWithPadding,\
    chooseScribbleFromMissedFGOrWrongCBG2D, chooseScribbleFromDefiniteRegion2D,\
    autoGenerateScribbleRegions2D,\
    chooseScribbleFromMissedFGOrWrongCBG3D, chooseScribbleFromDefiniteRegion3D,\
    autoGenerateScribbleAndBBox3D,\
    createGCInputUsingGT
# import src.gcHelper as gcHelper

#### Config

In [3]:
#Configs
patDataConfig = \
{   
     "ctSuffix": "_ct.nii.gz",
     "ptSuffix": "_pt.nii.gz",
     "gtSuffix": "_ct_gtvt.nii.gz",
     "predSuffix": "_segment.nii.gz",
     "softmaxSuffix": "_softMax.nii.gz",
     "fgScribbleSuffix": "_fgScribble.nii.gz",
     "bgScribbleSuffix": "_bgScribble.nii.gz",
     "ct_gCutSuffix": "_ct_gCut.nii.gz",
     "pet_gCutSuffix": "_pet_gCut.nii.gz",         
     "softmax_gCutSuffix": "_softmax_gCut.nii.gz",
     "ct_low": -1000,
     "ct_high": 3095,
     "ct_clipImages": False,
     "ct_scaleImages": False,
     "ct_scaleFactor": 1000,
     "ct_normalizeImages": False,
     "pt_low": -1000,
     "pt_high": 3095,
     "pt_clipImages": False,
     "pt_scaleImages": False,
     "pt_scaleFactor": 1,
     "pt_normalizeImages": False,
     "labels_to_train": [1]
}


autoScribbleAndGCConfig = \
{
 'bbPad' : 2,
 'fractionDivider' : 10,
 'dilation_diam' : 2,
 'useAtmostNScribblesPerRegion' : 4,
 'segparams_ssgc' : 
        {
         "method": "graphcut",
         "pairwise_alpha": 1.0,
         "modelparams": 
            {
             "cvtype": "full",
             "params": 
                {
                 "covariance_type": "full",
                 "n_components": 1
                }
            },
         "return_only_object_with_seeds": True
        }     
}

#### Basic input

In [4]:
srcFolder =\
'/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/validation_gtvs_withSoftmax'
expFolder = '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment'
# srcFolder = 'J:/HecktorData/nnUnet_3dfullres/validation_gtvs_withSoftmax'
# expFolder = 'J:/PlayDataManualSegmentation/AutoScribbleExperiment'
patientName = 'CHUM038'
expPatName = "expPat"

####  Prepare initial random scribble and run graphcut on CT / PET / Softmax (remotely as we are in slicer 4.11 environment  where imcut() could not be implemented)

In [160]:
print('Running single experiment on individual patient')

ctData,  ptData, gtData, softmaxData, predFromNN,\
    binLimit, bbVolume, numFGS, numBGS,\
    fgScribbleFromFGMissed, bgScribbleFromBGWrongC,\
    fgScribbleFromDefiniteFG, bgScribbleFromDefiniteBG,\
    fgScribble, bgScribble,\
    graphCutInputConfig, graphCutInputConfig_JsonFilePath\
   =  createGCInputUsingGT(patientName=patientName, srcFolder=srcFolder,\
        expFolder=expFolder, expPatName = expPatName,\
        patDataConfig=patDataConfig,\
        autoScribbleAndGCConfig = autoScribbleAndGCConfig,\
        verbose=verbose)

from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson    
gcAndDiceResult = remote_generateGrahcutSegmentationAndDiceFromJson(graphCutInputConfig_JsonFilePath)
print(gcAndDiceResult)

Running single experiment on individual patient
Starting action :  imcut
starting connection to ('127.0.0.1', 65432)
sending b'\x00h{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 127}{"action": "imcut", "value": "/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/graphCutInputConfig.json"}' to ('127.0.0.1', 65432)
received text/json response from ('127.0.0.1', 65432)
graphCut success : True
closing connection to ('127.0.0.1', 65432)
{'successFlag': True, 'patientName': 'CHUM038', 'gcPath_ct': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_ct_gCut.nii.gz', 'gcPath_pet': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_pet_gCut.nii.gz', 'gcPath_softmax': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_softmax_gCut.nii.gz', 'originalDice': 0.48517890685847803, 'numFGS': 8, 'numBGS': 8, 'gcDice_ct': 0.3803161195366993, 'gcDice

In [6]:
graphCutInputConfig_JsonFilePath = "/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/graphCutInputConfig.json"
from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson    
gcAndDiceResult = remote_generateGrahcutSegmentationAndDiceFromJson(graphCutInputConfig_JsonFilePath)
print(gcAndDiceResult)

Starting action :  imcut
starting connection to ('127.0.0.1', 65432)
sending b'\x00h{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 127}{"action": "imcut", "value": "/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/graphCutInputConfig.json"}' to ('127.0.0.1', 65432)
received text/json response from ('127.0.0.1', 65432)
graphCut success : True
closing connection to ('127.0.0.1', 65432)
{'successFlag': True, 'patientName': 'CHUM038', 'gcPath_ct': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_ct_gCut.nii.gz', 'gcPath_pet': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_pet_gCut.nii.gz', 'gcPath_softmax': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_softmax_gCut.nii.gz', 'originalDice': 0.48517890685847803, 'numFGS': 8, 'numBGS': 8, 'gcDice_ct': 0.3822694948353578, 'gcDice_pet': 0.4741745584847709, 'gcDice_softmax': 0.5

In [7]:
ctFileName = expPatName + patDataConfig['ctSuffix']
ptFileName = expPatName + patDataConfig['ptSuffix']
gtFileName = expPatName + patDataConfig['gtSuffix']
softmaxFileName = expPatName + patDataConfig['softmaxSuffix']
predFromNNFileName = expPatName + patDataConfig['predSuffix']
fgScribbleFileName = expPatName + patDataConfig['fgScribbleSuffix']
bgScribbleFileName = expPatName + patDataConfig['bgScribbleSuffix']
gc_ct_FileName = expPatName + patDataConfig['ct_gCutSuffix']
gc_pet_FileName = expPatName + patDataConfig['pet_gCutSuffix']
gc_softmax_FileName = expPatName + patDataConfig['softmax_gCutSuffix']
print(expFolder)
print(ctFileName)
print(ptFileName)
print(gtFileName)
print(softmaxFileName)
print(predFromNNFileName)
print(fgScribbleFileName)
print(bgScribbleFileName)
print(gc_ct_FileName)
print(gc_pet_FileName)
print(gc_softmax_FileName)

/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment
expPat_ct.nii.gz
expPat_pt.nii.gz
expPat_ct_gtvt.nii.gz
expPat_softMax.nii.gz
expPat_segment.nii.gz
expPat_fgScribble.nii.gz
expPat_bgScribble.nii.gz
expPat_ct_gCut.nii.gz
expPat_pet_gCut.nii.gz
expPat_softmax_gCut.nii.gz


In [8]:
import JupyterNotebooksLib as slicernb
slicernb.ViewDisplay('OneUpRedSlice') #'FourUp', 'OneUp3D'
slicer.mrmlScene.Clear()

In [9]:
#This one creates separate segment from  label file and attach to common segmentation node
def createSegmentFromLabelFile(folderPath, labelFileName, labelMapName,\
        parentSegmentationNode, colorTriple, sliceFillFlag, overallVisibility):
    #labelMapNode = slicer.util.addVolumeFromArray(labelData,ijkToRAS=None,
    #                      name=labelMapName,nodeClassName='vtkMRMLScalarVolumeNode')
    labelMapNode = slicer.util.loadLabelVolume(os.path.join(folderPath, labelFileName),\
            properties = {'name': labelMapName})
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelMapNode, parentSegmentationNode)
    slicer.mrmlScene.RemoveNode(labelMapNode)
    parentSegmentationNode.GetSegmentation().GetSegment(labelMapName).SetColor(colorTriple[0], colorTriple[1],colorTriple[2])
    parentSegmentationNode.GetDisplayNode().SetSegmentVisibility2DFill(labelMapName,sliceFillFlag)
    parentSegmentationNode.GetDisplayNode().SetSegmentVisibility(labelMapName,overallVisibility)
    return labelMapName

In [10]:
ctVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ctFileName))
ctVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeGrey')

ptVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ptFileName))
ptVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLPETProceduralColorNodePET-Heat')

softmaxVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,softmaxFileName))
softmaxVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeBlue')

original_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "original")
gt_Name = createSegmentFromLabelFile(expFolder, gtFileName, "gtLabelMap",\
          original_segmentationNode, (0,0,1), False, True)
predNN_Name = createSegmentFromLabelFile(expFolder, predFromNNFileName, "predNNLabelMap",\
          original_segmentationNode, (1,0,0), False, False)


fgScribble_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "fgScribble")
fgScribble_Name = createSegmentFromLabelFile(expFolder, fgScribbleFileName,  "fgScribbleLabelMap",\
          fgScribble_segmentationNode, (0.15, 0.7, 0.8), True, True)

bgScribble_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "bgScribble")
bgScribble_Name = createSegmentFromLabelFile(expFolder, bgScribbleFileName, "bgScribbleLabelMap",\
          bgScribble_segmentationNode, (0.8, 0.7, 0.15), True, True)

gcut_segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "graphcut")
gc_softmax_Name = createSegmentFromLabelFile(expFolder, gc_softmax_FileName, "gc_softmaxLabelMap",\
   gcut_segmentationNode, (0.3, 0.5, 0.7), False, True)    
gc_ct_Name = createSegmentFromLabelFile(expFolder, gc_ct_FileName, "gc_ctLabelMap",\
   gcut_segmentationNode, (0.7, 0.3, 0.5), False, False)    
gc_pet_Name = createSegmentFromLabelFile(expFolder, gc_pet_FileName, "gc_petLabelMap",\
   gcut_segmentationNode, (0.5, 0.7, 0.3), False, False)

# slicer.util.setSliceViewerLayers(background=ctVolumeNode, foreground=ptVolumeNode, label=gt_segmentationNode\
#                                 foregroundOpacity=0.5,labelOpacity=0.5)
slicer.util.setSliceViewerLayers(background=softmaxVolumeNode, foreground=ptVolumeNode, foregroundOpacity=0.5)

#reinforce visibility false:
original_segmentationNode.GetDisplayNode().SetSegmentVisibility(predNN_Name,False)
gcut_segmentationNode.GetDisplayNode().SetSegmentVisibility(gc_ct_Name,False)
gcut_segmentationNode.GetDisplayNode().SetSegmentVisibility(gc_pet_Name,False)


In [15]:
window = qt.QWidget()
window.setWindowTitle('QHBoxLayout')
layout = qt.QHBoxLayout()
layout.addWidget(qt.QPushButton('Left'))
layout.addWidget(qt.QPushButton('Center'))
layout.addWidget(qt.QPushButton('Right'))
window.setLayout(layout)
window.show()

In [186]:
def updateScribbleFile(scribble_segmentationNode, scribble_Name, masterVolumeNode,\
                       folderPath, scribbleFileName):
    #Make sure the segment is visible; Since this is only segment no need to make others invisible
    scribble_segmentationNode.GetDisplayNode().SetSegmentVisibility(scribble_Name,True)
    #set references to masterVolumeNode
    scribble_segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(masterVolumeNode)
    #make temp labelvolumenode
    temp_labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    #export Segmentation to labelmapNode
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(scribble_segmentationNode,\
    temp_labelmapVolumeNode, masterVolumeNode)
    #save labelmap to disk; before that copy original scribble to "old_" scribble file
    shutil.copy(os.path.join(folderPath, scribbleFileName), \
                os.path.join(folderPath, "old_"+ scribbleFileName))
    slicer.util.saveNode(temp_labelmapVolumeNode,os.path.join(folderPath, scribbleFileName))
    #Finally delete the temp labelvolumenode 
    slicer.mrmlScene.RemoveNode(temp_labelmapVolumeNode)

########################## Save update FG_scribble ########################
updateScribbleFile(fgScribble_segmentationNode, fgScribble_Name, softmaxVolumeNode,\
                       expFolder, fgScribbleFileName) 

########################## Save update BG_scribble ########################
updateScribbleFile(bgScribble_segmentationNode, bgScribble_Name, softmaxVolumeNode,\
                       expFolder, bgScribbleFileName) 



In [187]:
#Re run graphcut updated scribble 
from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson    
gcAndDiceResult = remote_generateGrahcutSegmentationAndDiceFromJson(graphCutInputConfig_JsonFilePath)
print(gcAndDiceResult)

Starting action :  imcut
starting connection to ('127.0.0.1', 65432)
sending b'\x00h{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 127}{"action": "imcut", "value": "/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/graphCutInputConfig.json"}' to ('127.0.0.1', 65432)
received text/json response from ('127.0.0.1', 65432)
graphCut success : True
closing connection to ('127.0.0.1', 65432)
{'successFlag': True, 'patientName': 'CHUM038', 'gcPath_ct': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_ct_gCut.nii.gz', 'gcPath_pet': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_pet_gCut.nii.gz', 'gcPath_softmax': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_softmax_gCut.nii.gz', 'originalDice': 0.48517890685847803, 'numFGS': 8, 'numBGS': 8, 'gcDice_ct': 0.3822694948353578, 'gcDice_pet': 0.4741745584847709, 'gcDice_softmax': 0.5

In [16]:
slicer.mrmlScene.Clear()

In [ ]:
# #Save scribble
# Export a selection of segments (identified by their names):
# segmentNames = ["Prostate", "Urethra"]
# segmentIds = vtk.vtkStringArray()
# for segmentName in segmentNames:
#     segmentId = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(segmentName)
#     segmentIds.InsertNextValue(segmentId)
# slicer.vtkSlicerSegmentationsModuleLogic.ExportSegmentsToLabelmapNode(segmentationNode,\
#             segmentIds, labelmapVolumeNode, referenceVolumeNode)

########################## Save update FG_scribble ########################
# fgScribble_segmentationNode.GetDisplayNode().SetSegmentVisibility(fgScribble_Name,True)
# #set references to masterVolumeNode
# fgScribble_segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(softmaxVolumeNode)
# #make temp labelvolumenode
# temp_labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
# #export Segmentation to labelmapNode
# slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(fgScribble_segmentationNode,\
#     temp_labelmapVolumeNode, softmaxVolumeNode)
# #save labelmap to disk; before that copy original scribble to "old_" scribble file
# shutil.copy(os.path.join(expFolder, fgScribbleFileName), \
#                 os.path.join(expFolder, "old_"+fgScribbleFileName))
# slicer.util.saveNode(temp_labelmapVolumeNode,os.path.join(expFolder, fgScribbleFileName))
# #Finally delete the temp labelvolumenode 
# slicer.mrmlScene.RemoveNode(temp_labelmapVolumeNode)

########################## Save update BG_scribble ########################
# bgScribble_segmentationNode.GetDisplayNode().SetSegmentVisibility(bgScribble_Name,True)
# #set references to masterVolumeNode
# bgScribble_segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(softmaxVolumeNode)
# #make temp labelvolumenode
# temp_labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
# #export Segmentation to labelmapNode
# slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(bgScribble_segmentationNode,\
#     temp_labelmapVolumeNode, softmaxVolumeNode)
# #save labelmap to disk
# shutil.copy(os.path.join(expFolder, bgScribbleFileName), \
#                 os.path.join(expFolder, "old_"+bgScribbleFileName))
# slicer.util.saveNode(temp_labelmapVolumeNode,os.path.join(expFolder, bgScribbleFileName))
# #Finally delete the temp labelvolumenode 
# slicer.mrmlScene.RemoveNode(temp_labelmapVolumeNode)
###########################################################################

In [ ]:
# #This one creates separate segmentation nodes from each label file : Not used any more
# def createSegmentationNodeFromLabelFile(folderPath, labelFileName, labelMapName,\
#         segmentationNodeName, colorTriple, sliceFillFlag, overallVisibility=True):
#     #labelMapNode = slicer.util.addVolumeFromArray(labelData,ijkToRAS=None,
#     #                      name=labelMapName,nodeClassName='vtkMRMLScalarVolumeNode')
#     labelMapNode = slicer.util.loadLabelVolume(os.path.join(folderPath, labelFileName),\
#             properties = {'name': labelMapName})
#     segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", segmentationNodeName)
#     slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelMapNode, segmentationNode)
#     slicer.mrmlScene.RemoveNode(labelMapNode)
#     segmentationNode.GetSegmentation().GetSegment(labelMapName).SetColor(colorTriple[0], colorTriple[1],colorTriple[2])
#     if True == sliceFillFlag:
#         segmentationNode.GetDisplayNode().SetVisibility2DFill(1)
#         #segmentationNode.SetSegmentVisibility2DFill('Segment_1',True)
#     else:
#         segmentationNode.GetDisplayNode().SetVisibility2DFill(0)
#         #segmentationNode.SetSegmentVisibility2DFill('Segment_1',False)
#     #set overall visibility
#     segmentationNode.GetDisplayNode().SetVisibility(overallVisibility)
#     return segmentationNode

In [ ]:
# ctVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ctFileName))
# ctVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeGrey')

# ptVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ptFileName))
# ptVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLPETProceduralColorNodePET-Heat')

# softmaxVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,softmaxFileName))
# softmaxVolumeNode.GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLColorTableNodeBlue')

# gt_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, gtFileName, \
#                          "gtLabelMap", "gtSeg", (0,0,1), False, True)
# predNN_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, predFromNNFileName, \
#                          "predNNLabelMap", "predNNSeg", (1,0,0), False, False)

# fgScribble_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, fgScribbleFileName, \
#                          "fgScribbleLabelMap", "fgScribbleSeg", (0.15, 0.7, 0.8), True, True)
# bgScribble_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, bgScribbleFileName, \
#                          "bgScribbleLabelMap", "bgScribbleSeg", (0.8, 0.7, 0.15), True, True)

# gc_softmax_segmentationNode = createSegmentationNodeFromLabelFile(expFolder,\
#   gc_softmax_FileName, "gc_softmaxLabelMap", "gc_softmaxSeg", \
#     (0.3, 0.5, 0.7), False, True)

# gc_ct_segmentationNode = createSegmentationNodeFromLabelFile(expFolder,\
#   gc_ct_FileName, "gc_ctLabelMap", "gc_ctSeg", \
#     (0.7, 0.3, 0.5), False, False)

# gc_pet_segmentationNode = createSegmentationNodeFromLabelFile(expFolder,\
#   gc_pet_FileName, "gc_petLabelMap", "gc_petSeg", \
#     (0.5, 0.7, 0.3), False, False)

# # slicer.util.setSliceViewerLayers(background=ctVolumeNode, foreground=ptVolumeNode, label=gt_segmentationNode\
# #                                 foregroundOpacity=0.5,labelOpacity=0.5)
# slicer.util.setSliceViewerLayers(background=softmaxVolumeNode, foreground=ptVolumeNode, foregroundOpacity=0.5)

In [ ]:
# gt_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, gtFileName, \
#                          "gtLabelMap", "gtSeg", (0,0,1), False)
# predNN_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, predFromNNFileName, \
#                          "predNNLabelMap", "predNNSeg", (1,0,0), False)
# fgScribble_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, fgScribbleFileName, \
#                          "fgScribbleLabelMap", "fgScribbleSeg", (0.15, 0.7, 0.8), True)
# bgScribble_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, bgScribbleFileName, \
#                          "bgScribbleLabelMap", "bgScribbleSeg", (0.8, 0.7, 0.15), True)

# # slicer.util.setSliceViewerLayers(background=ctVolumeNode, foreground=ptVolumeNode, label=gt_segmentationNode\
# #                                 foregroundOpacity=0.5,labelOpacity=0.5)


In [ ]:
#Python interactor output
# >>> expPat_pt_VolumeNode = getNode('expPat_pt')
# >>> type(expPat_pt_VolumeNode)
# <class 'MRMLCorePython.vtkMRMLScalarVolumeNode'>
# >>> expPat_pt_DisplayNode = expPat_pt_VolumeNode.GetDisplayNode()
# >>> type(expPat_pt_DisplayNode)
# <class 'MRMLCorePython.vtkMRMLScalarVolumeDisplayNode'>
# >>> expPat_pt_ColorNode = expPat_pt_DisplayNode.GetColorNode()
# >>> type(expPat_pt_ColorNode)
# <class 'MRMLCorePython.vtkMRMLColorTableNode'>
# >>> expPat_pt_CurrentLUT = expPat_pt_ColorNode.GetLookupTable()
# >>> type(expPat_pt_CurrentLUT)
# <class 'vtkCommonCorePython.vtkLookupTable'>

# >>> indexL = expPat_pt_CurrentLUT.GetIndexedLookup()
# >>> type(indexL)
# <class 'int'>
# >>> print(indexL)
# 0
# >>> expPat_pt_CurrentLUT.SetIndexedLookup(3)
# >>> expPat_pt_CurrentLUT = expPat_pt_ColorNode.GetLookupTable()
# >>> print(expPat_pt_CurrentLUT.GetIndexedLookup())
# 3
# >>> getNode('expPat_pt').GetDisplayNode().GetColorNode().IsTypeOf('vtkMRMLColorTableNode')
# 1
# >>> getNode('expPat_pt').GetDisplayNode().GetColorNode().IsTypeOf('vtkMRMLProceduralColorNode')
# 0

In [ ]:
# #Experimental read write json file
# with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_graphCutInput.json") as fp:
#         gcSConfig = json.load(fp)
#         fp.close() 
# srcFolder = gcSConfig["srcFolder" ]
# patDataConfig = gcSConfig["patDataConfig" ]
# autoScribbleAndGCConfig = gcSConfig['autoScribbleAndGCConfig']
# expFolder = patDataConfig["expFolder"]

# print(srcFolder)
# print(expFolder)
# print(autoScribbleAndGCConfig)
# print(patDataConfig['labels_to_train'])

# #Make changes

# new_expFolder = "/home/user/DMML/Data/PlayDataManualSegmentation/ManualScribble"
# new_patDataConfig = patDataConfig
# new_patDataConfig["expFolder"]= new_expFolder

# new_srcFolder = "/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/training_gtvs_withSoftmax"
# new_gcSConfig = gcSConfig
# new_gcSConfig["srcFolder" ] = new_srcFolder
# new_gcSConfig["patDataConfig" ] = new_patDataConfig
# with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/modified_expPat_graphCutInput.json", 'w') as fp:
#         json.dump(new_gcSConfig, fp, ) #, indent='' #, indent=4
#         fp.close()
        
# #Let us test
# #Experimental read write json file
# with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/modified_expPat_graphCutInput.json") as fp:
#         gcSConfig2 = json.load(fp)
#         fp.close() 
# srcFolder2 = gcSConfig2["srcFolder" ]
# patDataConfig2 = gcSConfig2["patDataConfig" ]
# autoScribbleAndGCConfig2 = gcSConfig2['autoScribbleAndGCConfig']
# expFolder2 = patDataConfig2["expFolder"]

# print('####################')
# print(srcFolder2)
# print(expFolder2)
# print(autoScribbleAndGCConfig2)
# print(patDataConfig2['labels_to_train'])